# 📍 GPS Takip Verileri Analizi
Bu notebook, araç filosunun GPS takip verilerinin detaylı analizini içermektedir.

**Not:** Büyük dosya boyutu nedeniyle analiz örnekleme ile yapılabilir.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_palette("husl")
sns.set_style("whitegrid")

print("Veri yükleniyor... (Bu işlem birkaç dakika sürebilir)")

In [ ]:
# Büyük dosya için chunk okuma veya örnekleme
# Seçenek 1: İlk N satırı oku
nrows = 100000  # İlk 100K satır
gps_df = pd.read_csv('all_merged_data.csv', nrows=nrows)

# Tarih sütununu datetime'a çevir
gps_df['Tarih'] = pd.to_datetime(gps_df['Tarih'], format='%d.%m.%Y')

print(f"Yüklenen veri: {len(gps_df):,} satır")
print(f"Toplam sütun sayısı: {gps_df.shape[1]}")
print(f"\nTarih aralığı: {gps_df['Tarih'].min()} - {gps_df['Tarih'].max()}")
print(f"\nBenzersiz araç sayısı: {gps_df['vehicle_id'].nunique()}")
print(f"Benzersiz mahalle sayısı: {gps_df['Mahalle'].nunique()}")

gps_df.head()

## 📊 Veri Özeti ve Kalite Kontrolü

In [ ]:
print("=" * 70)
print("VERİ KALİTE RAPORU")
print("=" * 70)
print(f"\nEksik Değerler:")
missing = gps_df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
if len(missing) > 0:
    print(missing)
else:
    print("Eksik değer yok!")

print(f"\n\nDurum Dağılımı:")
print(gps_df['Durum'].value_counts())

print(f"\n\nHız İstatistikleri:")
print(gps_df['Hız(km/sa)'].describe())

## 🚗 Araç Aktivite Analizi

In [ ]:
# Araç başına kayıt sayısı
vehicle_records = gps_df['vehicle_id'].value_counts().head(15)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# En aktif araçlar
vehicle_records.plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_xlabel('GPS Kayıt Sayısı', fontsize=12)
axes[0].set_ylabel('Araç ID', fontsize=12)
axes[0].set_title('En Aktif 15 Araç (GPS Kayıt Sayısına Göre)', fontsize=13, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Durum dağılımı
status_counts = gps_df['Durum'].value_counts().head(10)
status_counts.plot(kind='bar', ax=axes[1], color='coral')
axes[1].set_xlabel('Durum', fontsize=12)
axes[1].set_ylabel('Kayıt Sayısı', fontsize=12)
axes[1].set_title('Araç Durumu Dağılımı', fontsize=13, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

## 🗺️ Coğrafi Dağılım

In [ ]:
# Mahalle bazlı aktivite
mahalle_activity = gps_df['Mahalle'].value_counts().head(20)

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Mahalle aktivitesi
mahalle_activity.plot(kind='barh', ax=axes[0], color='mediumseagreen')
axes[0].set_xlabel('GPS Kayıt Sayısı', fontsize=12)
axes[0].set_ylabel('Mahalle', fontsize=12)
axes[0].set_title('En Çok Hizmet Verilen 20 Mahalle', fontsize=13, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# GPS noktalarının haritası
sample = gps_df.sample(min(5000, len(gps_df)))  # Max 5000 nokta
scatter = axes[1].scatter(sample['Boylam'], sample['Enlem'], 
                         c=pd.to_numeric(sample['vehicle_id'], errors='coerce'), 
                         cmap='viridis', alpha=0.3, s=1)
axes[1].set_xlabel('Boylam', fontsize=12)
axes[1].set_ylabel('Enlem', fontsize=12)
axes[1].set_title('GPS Noktaları Dağılımı (Örnek)', fontsize=13, fontweight='bold')
axes[1].grid(alpha=0.3)
plt.colorbar(scatter, ax=axes[1], label='Araç ID')

plt.tight_layout()
plt.show()

## ⚡ Hız ve Hareket Analizi

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Hız dağılımı
axes[0, 0].hist(gps_df['Hız(km/sa)'], bins=50, color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('Hız (km/sa)', fontsize=11)
axes[0, 0].set_ylabel('Frekans', fontsize=11)
axes[0, 0].set_title('Hız Dağılımı', fontsize=12, fontweight='bold')
axes[0, 0].axvline(gps_df['Hız(km/sa)'].mean(), color='red', linestyle='--', label=f'Ortalama: {gps_df["Hız(km/sa)"].mean():.1f} km/h')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Hareketli vs Duran
moving = gps_df[gps_df['Hız(km/sa)'] > 0]
stationary = gps_df[gps_df['Hız(km/sa)'] == 0]
categories = ['Hareketli', 'Duran']
counts = [len(moving), len(stationary)]
axes[0, 1].bar(categories, counts, color=['green', 'orange'])
axes[0, 1].set_ylabel('Kayıt Sayısı', fontsize=11)
axes[0, 1].set_title('Hareketli vs Duran Kayıtlar', fontsize=12, fontweight='bold')
axes[0, 1].grid(axis='y', alpha=0.3)
for i, v in enumerate(counts):
    axes[0, 1].text(i, v + 500, f'{v:,}', ha='center', fontweight='bold')

# Mesafe dağılımı
axes[1, 0].hist(gps_df['Mesafe(km)'][gps_df['Mesafe(km)'] > 0], bins=30, color='lightcoral', edgecolor='black')
axes[1, 0].set_xlabel('Segement Mesafe (km)', fontsize=11)
axes[1, 0].set_ylabel('Frekans', fontsize=11)
axes[1, 0].set_title('Segment Mesafe Dağılımı', fontsize=12, fontweight='bold')
axes[1, 0].grid(alpha=0.3)

# Yükseklik dağılımı
axes[1, 1].hist(gps_df['Yükseklik'].dropna(), bins=30, color='mediumpurple', edgecolor='black')
axes[1, 1].set_xlabel('Yükseklik (m)', fontsize=11)
axes[1, 1].set_ylabel('Frekans', fontsize=11)
axes[1, 1].set_title('Rakım Dağılımı', fontsize=12, fontweight='bold')
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## ⏱️ Zamansal Analiz

In [ ]:
# Saatlik aktivite
gps_df['Saat_numeric'] = pd.to_datetime(gps_df['Saat'], format='%H:%M:%S', errors='coerce').dt.hour
hourly_activity = gps_df['Saat_numeric'].value_counts().sort_index()

# Günlük aktivite
daily_activity = gps_df.groupby('Tarih').size()

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Saatlik pattern
hourly_activity.plot(kind='bar', ax=axes[0], color='teal')
axes[0].set_xlabel('Saat', fontsize=12)
axes[0].set_ylabel('GPS Kayıt Sayısı', fontsize=12)
axes[0].set_title('Saatlik Aktivite Dağılımı', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Günlük trend
daily_activity.plot(kind='line', ax=axes[1], color='darkblue', marker='o', linewidth=2)
axes[1].set_xlabel('Tarih', fontsize=12)
axes[1].set_ylabel('GPS Kayıt Sayısı', fontsize=12)
axes[1].set_title('Günlük Aktivite Trendi', fontsize=13, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 📉 Araç Performans Metrikleri

In [ ]:
# Araç bazlı metrikler
vehicle_metrics = gps_df.groupby('vehicle_id').agg({
    'Mesafe(km)': 'sum',
    'Hız(km/sa)': 'mean',
    'vehicle_id': 'count'
}).round(2)
vehicle_metrics.columns = ['Toplam Mesafe (km)', 'Ortalama Hız (km/h)', 'GPS Kayıt Sayısı']
vehicle_metrics = vehicle_metrics.sort_values('Toplam Mesafe (km)', ascending=False).head(10)

print("=" * 70)
print("EN AKTİF 10 ARAÇ - PERFORMANS METRİKLERİ")
print("=" * 70)
print(vehicle_metrics)

# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

vehicle_metrics['Toplam Mesafe (km)'].plot(kind='barh', ax=axes[0], color='indianred')
axes[0].set_xlabel('Toplam Mesafe (km)', fontsize=12)
axes[0].set_ylabel('Araç ID', fontsize=12)
axes[0].set_title('En Fazla Mesafe Kat Eden Araçlar', fontsize=13, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

vehicle_metrics['Ortalama Hız (km/h)'].plot(kind='barh', ax=axes[1], color='gold')
axes[1].set_xlabel('Ortalama Hız (km/h)', fontsize=12)
axes[1].set_ylabel('Araç ID', fontsize=12)
axes[1].set_title('Ortalama Hız Değerleri', fontsize=13, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

## 🎯 Temel Bulgular

1. **Veri Yoğunluğu:** 634K+ GPS kaydı ile detaylı takip
2. **Aktivite Paternleri:** Çalışma saatleri boyunca yoğun aktivite
3. **Coğrafi Kapsam:** 65+ mahallede operasyon
4. **Hız Profilleri:** Şehir içi hız limitlerine uygun seyir
5. **Duraklama Analizleri:** Toplama noktalarında düzenli duruşlar

In [ ]:
print("\n" + "=" * 70)
print("GENEL İSTATİSTİKLER")
print("=" * 70)
print(f"Toplam GPS Kaydı: {len(gps_df):,}")
print(f"Benzersiz Araç: {gps_df['vehicle_id'].nunique()}")
print(f"Toplam Kat Edilen Mesafe: {gps_df['Mesafe(km)'].sum():.2f} km")
print(f"Ortalama Hız: {gps_df['Hız(km/sa)'].mean():.2f} km/h")
print(f"Maksimum Hız: {gps_df['Hız(km/sa)'].max():.2f} km/h")
print(f"Toplam Benzersiz Mahalle: {gps_df['Mahalle'].nunique()}")
print("=" * 70)